<a href="https://colab.research.google.com/github/DarkNightSoldier/DTApp/blob/main/ProyectoBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tabula-py
!pip install pandasql
!pip install python-docx
!wget https://raw.githubusercontent.com/DarkNightSoldier/DTApp/main/DatosApp.db


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [1]:
from funciones_estudio import *

for i in range(0,16):
    print(Realizar_Estudio(f'HistAcad/{i}.pdf'))

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Estudios/Pepito Perez Daza_estudio_dt.docx
Estudios/Adelaida Bejarano Manotas_estudio_dt.docx


KeyboardInterrupt: 

In [ ]:
Actualizar_Historia("/HistAcad/06.pdf","/HistAcad/06.pdf")